In [1]:
import os
import ee
import geemap
from pathlib import Path
from datetime import datetime, timedelta , date
import time
import pandas as pd
import csv
import glob
import folium
from IPython.display import display, clear_output
import shutil
import shapefile
import ipywidgets as widgets
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
shp = shapefile.Reader("".join(my_button.files))
b = []
for i in range(len(shp.fields)):
    a = shp.fields[i][0]
    b.append(a)

# give the shapefile name

file_name = widgets.Dropdown(
options=b,
description='Regional category')

file_name

Dropdown(description='Regional category', options=('DeletionFlag', 'STATE'), value='DeletionFlag')

In [6]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [7]:
# give CLOUDY_PIXEL_PERCENTAGE

CLOUDY_PIXEL_PERCENTAGE = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=10,
    description='CLOUDY_PIXEL_PERCENTAGE:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

CLOUDY_PIXEL_PERCENTAGE

IntSlider(value=20, continuous_update=False, description='CLOUDY_PIXEL_PERCENTAGE:', step=10)

In [8]:
# give the bands
#
band_name = widgets.SelectMultiple(
options=['NDVI'],
description='Band',
)

band_name

SelectMultiple(description='Band', options=('NDVI',), value=())

In [9]:
statics =widgets.Dropdown(
    options=['MEAN','MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'],
    value='MEAN',
    description='Statistics')

statics

Dropdown(description='Statistics', options=('MEAN', 'MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'),…

In [10]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\Yang Hsiu\Downloads\satellite python code-20220518T062225Z-001\satellite python cod…

In [11]:
# create folder
folder_name = 'data_all'

# create folder name : data_all
if os.path.isdir(folder_name) == True:
    shutil.rmtree(folder_name, ignore_errors=True)
    os.makedirs(folder_name)
else:
    os.makedirs(folder_name)

In [12]:
states = geemap.shp_to_ee("".join(my_button.files))

def maskS2clouds(image):
    qa = image.select('QA60');
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask).divide(10000)


# get indexes
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)


def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def trans_date(input_date):
    t1 = input_date
    t2 = datetime.strptime(t1, '%Y%m%d').strftime('%Y/%m/%d')
    return t2


def cbind(statics):

    all_files = glob.glob(os.path.join(folder_name,"Sentinel-2_{}*.csv".format(statics)))

    df_from_each_file = (pd.read_csv(f, sep = ",") for f in all_files)
    df_merged = pd.concat(df_from_each_file, ignore_index = True)
    if 'ndvi' in df_merged.columns.tolist():
        df_merged.rename(columns={'NDVI' : 'NDVI_' + str(statics)}, inplace = True)
    else:
        pass
    
    df_merged = df_merged.groupby(['Date','Doy',file_name.value]).mean().reset_index()
   
    df_merged.to_csv(folder.selected + '.csv')
    
    shutil.rmtree(folder_name, ignore_errors=True)

""" 
Note that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, 
so this time the following line of code is to output the files to another folder.
"""


' \nNote that here, the merged file path cannot be the same as the single file path, and must be placed in different folders, \nso this time the following line of code is to output the files to another folder.\n'

In [13]:
def zonal(statics):
    time_list =monthlist(star.value, end.value)
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        if os.path.isfile(folder_name +'Sentinel-2_{}_{}.csv'.format(statics,time_list[i])) == True:
                
                continue
            
        else:
            clear_output(wait=True)
            Landsat = ee.ImageCollection('COPERNICUS/S2') \
                      .filterDate(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1)) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUDY_PIXEL_PERCENTAGE.value)) \
                      .map(maskS2clouds) \
                      .map(getNDVI) \
                      .filterBounds(states) \
                      .select(list(band_name.value)) \
                      .map(lambda image: image.clip(states))


            Landsat = Landsat.toBands()
            out_dir = os.path.expanduser(folder_name)
            out_dem_stats = os.path.join(out_dir, 'Sentinel-2_{}_{}.csv'.format(statics,time_list[i]))

            if not os.path.exists(out_dir):
                os.makedirs(out_dir)

            geemap.zonal_statistics(Landsat, states, out_dem_stats, statistics_type=statics, scale=1000)
            data_temp = pd.read_csv(out_dem_stats)
            column_name_list = data_temp.columns.tolist()
            c = []
            d = []
            for k in zip(column_name_list[:]):
                c.append(k[0][0])
                d.append(k[0])
            data = []
            for j in range(0, len(column_name_list),len(band_name.value)):            
                if all(m.isdigit() for m in c[j:j+len(band_name.value)]) == True:
                        
                    df = data_temp.loc[:,d[j:j+len(band_name.value)]]
                    df[file_name.value] = data_temp.loc[:,[file_name.value]]
                    text = column_name_list[j][0:8]
                    df.insert(0, 'Date', '')
                    df['Date'] = trans_date(text)
                    df.insert(1, 'Doy', '')
                    df['Doy'] = datetime.strptime(text, '%Y%m%d').strftime('%j')
                    colnames=['Date','Doy']
                    colnames.extend(list(band_name.value))
                    colnames.append(file_name.value)
                    df.columns=[colnames]
                    data.append(df)
                else:
                    continue
                        
            appended_data = pd.concat(data, axis=0,ignore_index = True)
            #cols = appended_data.columns.to_list()
            #cols.insert(len(appended_data.columns), cols.pop(cols.index(file_name)))
            #appended_data = appended_data[cols]
            appended_data.to_csv(out_dem_stats,index=False)#Output the file with date and doy back
            
            end_time = time.time()
            print(end_time - star_time)
            time.sleep(1)

In [ ]:
zonal(statics.value)
cbind(statics.value)

Computing statistics ...
